In [1]:
import pandas as pd
from IPython.display import display_markdown



In [2]:
file_name = 'Climate Master.xlsx'
climate_master = pd.read_excel(file_name)

In [3]:
# Helper methods

def extractColumn(
    column_name: str, 
    master_data: pd.DataFrame = None,
    rename: str = 'name'
) -> pd.DataFrame:
    
    """
    Extract a dataframe by given column and 
    rename the column to name by default.
    """
    
    if master_data is None:
        master_data = climate_master

    result = master_data[[column_name]]
    result = result.drop_duplicates()
    result = result.dropna()
    result = result.reset_index(drop=True)

    if rename:
        result = result.rename(columns={column_name: rename})
        
    return result


def extractColumnDuplicates(
    data: pd.DataFrame,
    column_name: str = 'name'
) -> pd.DataFrame :
    
    """
    Extract duplicated names in the given dataframe.
    """

    return data[data.duplicated(subset=column_name)]

def addConstantColumn(
    data: pd.DataFrame, 
    column_name: str, 
    column_value
) -> pd.DataFrame:
    
    """
    Add a new constant column with a given column 
    name and value.
    """
    
    data = data.copy()
    data[column_name] = column_value
    return data

def extractInvestors() -> pd.DataFrame:

    """
    To extract investors from the master dataframe.
    """

    investors = extractColumn('Investor')
    formatted_investors = []
    for group_investors in investors['name']:
        group_investors = str(group_investors)
        if ',' in group_investors:
            formatted_investors += [ investor.strip() for investor in group_investors.split(',')]
        else:
            formatted_investors.append(group_investors)

    formatted_investors.sort()

    investors = extractColumn('name', pd.DataFrame(data = formatted_investors, columns = ['name']))
    investors


In [4]:
# Extracting different entities
priorities = extractColumn('Priorities')
verticals = extractColumn('Vertical')
business_focus = extractColumn('Business Focus')
sectors = extractColumn('Sector')
industries = extractColumn(
    'name',
    pd.concat([
        extractColumn('Industry'), 
        extractColumn('Sub-Industry')
    ]),
    False
)
technologies = extractColumn('Technologies')
funding_rounds = extractColumn('Latest founding round')
investors = extractInvestors()


In [5]:
# Generate dataframe to visualize intersects between
# Priorities, Verticals, Business Focus, Sectors, Industries,
# & Technologies
column_name = 'table'
different_attributes = pd.concat([
    addConstantColumn(priorities, column_name, 'priorities'),
    addConstantColumn(verticals, column_name, 'verticals'),
    addConstantColumn(business_focus, column_name, 'business_focus'),
    addConstantColumn(sectors, column_name, 'sectors'),
    addConstantColumn(industries, column_name, 'industries'),
    addConstantColumn(technologies, column_name, 'technologies')
]).reset_index(drop=True)

grouped_attributes = different_attributes.groupby(by=['name'])
duplicated_attributes = pd.DataFrame()
for k, df in grouped_attributes:
    if df.shape[0] > 1:
        duplicated_attributes = pd.concat([duplicated_attributes, df])

if duplicated_attributes.shape[0] > 0:
    unique_duplicated_names = duplicated_attributes['name'].drop_duplicates()
    unique_duplicated_attribute_names = duplicated_attributes['table'].drop_duplicates()

    attribute_question = "#### What's the definition or relationship between the following columns:\n"

    for att_name in unique_duplicated_attribute_names:
        attribute_question = attribute_question + "\n- " + att_name

    attribute_question += "\n\n#### There are intersects in the following column values:\n"

    for att_value in unique_duplicated_names:
        attribute_question = attribute_question + "\n- " + att_value

    display_markdown(attribute_question, raw=True)


#### What's the definition or relationship between the following columns:

- verticals
- business_focus
- industries
- technologies
- priorities

#### There are intersects in the following column values:

- Battery Swapping Systems
- Bio-energy
- EV Charging Networks
- EV Infrastructure Analytics
- Geothermal
- Hydrogen Fuel Cell Developers
- Ocean
- Reforestation
- Solar
- Wind

### Use this block to address questions above (if any):

In [6]:
# Dataframe view of duplicated attributes
duplicated_attributes

,name,table
238,Battery Swapping Systems,verticals
283,Battery Swapping Systems,business_focus
447,Bio-energy,industries
461,Bio-energy,technologies
235,EV Charging Networks,verticals
300,EV Charging Networks,business_focus
252,EV Infrastructure Analytics,verticals
296,EV Infrastructure Analytics,business_focus
26,Geothermal,verticals
466,Geothermal,technologies


In [7]:
# Extract dataframe with more than one occurance of company name
duplicate_companies = pd.DataFrame()
for company_name, company_df in climate_master.groupby(by="Company Name"):
    if company_df.shape[0] > 1:
        duplicate_companies = pd.concat([duplicate_companies, company_df])


# If there are rows that shares company name, generate excel file, duplicated_companies.xlsx
if duplicate_companies.shape[0] > 0:
    columns_to_drop = ['Description']
    duplicate_companies.drop(columns=columns_to_drop, axis=1).reset_index(drop=True).to_excel('duplicated_companies.xlsx')
    print("There are rows that shared the same company, please refer to duplicated_companies.xlsx")

There are rows that shared the same company, please refer to duplicated_companies.xlsx
